# CNN(Convolutional Neural Network)

텐서플로우로 아래 그림과 동일한 CNN을 직접 구현하고, MNIST 손글씨로 학습 및 테스트를 진행해보도록 한다.

In [6]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

### 데이터 획득

In [7]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

0부터 255까지의 그레이 스테일을 확인할 수 있다.

In [8]:
print(x_train.shape)

(60000, 28, 28)


In [11]:
print(x_train[0][8])

[  0   0   0   0   0   0   0  18 219 253 253 253 253 253 198 182 247 241
   0   0   0   0   0   0   0   0   0   0]


0부터 9까지의 이미지에 해당하는 숫자를 확인할 수 있다.

In [12]:
print(y_train[0:9])

[5 0 4 1 9 2 1 3 1]


In [14]:
print("test data has " + str(x_train.shape[0]) + " samples") 
print("every test data is " + str(x_test.shape[1]) + " * " + str(x_test.shape[2]) + " image")

test data has 60000 samples
every test data is 28 * 28 image


### 데이터 구조 변경하기
입력 레이어에 데이터를 넣기 위해서 데이터의 구조를 변경해준다.

In [15]:
import numpy as np
x_train = np.reshape(x_train, (60000,28,28,1))
x_test = np.reshape(x_test, (10000,28,28,1))

print(x_train.shape)
print(x_test.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)


### 데이터 정규화
데이터 정규화는 보통 학습 시간을 단축하고 더 나은 성능을 구하도록 도와준다.  
MNIST 데이터의 모든 값은 0부터 255 범위 안에 있으므로, 255로 나눠줌으로써 모든 값을 0부터 1사이의 값으로 정규화한다.

In [17]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

gray_scale = 255
x_train /= gray_scale
x_test /= gray_scale

### 실제값을 one hot encoding으로 계산하기
손실함수에서 크로스 엔트로피를 계산하기 위해 실제값은 one hot encoding 값으로 변경한다.

In [18]:
num_classes = 10
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

## CNN 텐서플로우로 구현하기

In [19]:
from IPython.display import Image
Image(url= "https://raw.githubusercontent.com/captainchargers/deeplearning/master/img/practice_cnn.png", width=800, height=200)

In [22]:
model = Sequential()
model.add(Conv2D(16, kernel_size=(5,5), activation='relu', input_shape=(28,28,1), padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32, kernel_size=(5,5), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

크로스 엔트로피와 아담 옵티마이저를 사용해 모델을 최적화시키도록 구성한다. 모델은 두번 이상 모델의 개선이 없을 경우 조기 종료한다.

In [23]:
model.compile(loss=categorical_crossentropy,
             optimizer=Adam(),
             metrics=['accuracy'])

In [25]:
callbacks = [EarlyStopping(monitor='val_accuracy', patience=2, restore_best_weights=False),
            ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True)]

학습 데이터의 10%를 검증 데이터로 사용

In [26]:
model.fit(x_train, y_train, 
          batch_size=500, 
          epochs=5, 
          verbose=1, 
          validation_split=0.1, 
          callbacks=callbacks)

Epoch 1/5
108/108 [==============================] - 14s 125ms/step - loss: 0.9633 - accuracy: 0.7331 - val_loss: 0.1258 - val_accuracy: 0.9653
Epoch 2/5
108/108 [==============================] - 14s 131ms/step - loss: 0.1295 - accuracy: 0.9629 - val_loss: 0.0654 - val_accuracy: 0.9818
Epoch 3/5
108/108 [==============================] - 14s 125ms/step - loss: 0.0738 - accuracy: 0.9776 - val_loss: 0.0551 - val_accuracy: 0.9828
Epoch 4/5
108/108 [==============================] - 13s 125ms/step - loss: 0.0528 - accuracy: 0.9837 - val_loss: 0.0446 - val_accuracy: 0.9870
Epoch 5/5
108/108 [==============================] - 13s 123ms/step - loss: 0.0424 - accuracy: 0.9864 - val_loss: 0.0456 - val_accuracy: 0.9882


In [27]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.0387878343462944
Test accuracy: 0.9871000051498413
